## Using TinyGrad:-

In [ ]:
!git clone https://github.com/geohot/tinygrad.git

Cloning into 'tinygrad'...
remote: Enumerating objects: 14499, done.
remote: Counting objects: 100% (1897/1897), done.
remote: Compressing objects: 100% (350/350), done.
remote: Total 14499 (delta 1711), reused 1611 (delta 1547), pack-reused 12602
Receiving objects: 100% (14499/14499), 18.41 MiB | 36.38 MiB/s, done.
Resolving deltas: 100% (10240/10240), done.


In [ ]:
cd tinygrad/

/content/tinygrad


In [ ]:
!python3 -m pip install -e .

Obtaining file:///content/tinygrad
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.2/919.2 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 8.4 MB/s eta 0:00:00
  Running setup.py develop for tinygrad


In [ ]:
pwd

'/content/tinygrad'

### TinyGrad with Mnist dataset

In [ ]:
import time, sys
import json
import tqdm
import numpy as np
from tinygrad.tensor import Tensor
from tinygrad.nn import optim
import tinygrad.nn as nn
from tinygrad.helpers import flatten
from tinygrad.nn.optim import SGD, Adam
from sklearn.datasets import fetch_openml
from tinygrad.state import safe_save, safe_load, get_state_dict, load_state_dict, torch_load

In [ ]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False, )
print(X.shape, y.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784) (70000,)


In [ ]:
class LeNetTinyNet:
    def __init__(self):
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = x.reshape((x.shape[0], 1, 28, 28))
        x = self.conv1(x)
        x = x.tanh()
        x = x.avg_pool2d(2)    ## Conv1:  (64, 6, 14, 14)

        x = self.conv2(x)
        x = x.tanh()
        x = x.avg_pool2d(2)    ## Conv2:  (64, 16, 5, 5)

        x = self.conv3(x)
        x = x.tanh()

        x = x.reshape((x.shape[0], -1))  ## Reshape:  (64, 120)

        x = self.fc1(x)  ## FC1:  (64, 84)
        x = x.tanh()

        x = self.fc2(x)   ## FC2:  (64, 10)
        return x.log_softmax()

net = LeNetTinyNet()

In [ ]:
Tensor.training = True

In [ ]:
# from extra.training import sparse_categorical_crossentropy
def cross_entropy(out, Y):
  num_classes = out.shape[-1]
  YY = Y.flatten().astype(np.int32)
  y = np.zeros((YY.shape[0], num_classes), np.float32)
  y[range(y.shape[0]),YY] = -1.0*num_classes
  y = y.reshape(list(Y.shape)+[num_classes])
  y = Tensor(y)
  return out.mul(y).mean()

In [ ]:
def get_weights(weights):
    # Get the float values of the weights as a NumPy array
    weights_array = weights.numpy().astype(float)

    # Alternatively, get the float values as a Python list
    weights_list = list(weights.numpy())
    return weights_list

In [ ]:
opt = Adam([net.conv1.weight, net.conv2.weight, net.conv3.weight], lr=3e-4)

In [ ]:
num_epochs = 2000

weight_bias_dictionary = {}
running_loss, correct, total = 0.0, 0.0, 0.0
for epoch in range(num_epochs):
    weigth_bias = {}
    start_time = time.time()

    # Randomly sample a batch
    samp = np.random.randint(0, X.shape[0], size=(64))
    batch = Tensor(X[samp].astype('float32') / 255.0, requires_grad=False)
    # Get the corresponding labels
    labels = y[samp]

    # Forward pass
    out = net.forward(batch)

    # Compute loss
    loss = cross_entropy(out, labels)

    # Zero gradients
    opt.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    opt.step()

    # Calculate accuracy
    pred = np.argmax(out.numpy(), axis=-1)
    labels = [eval(label) for label in labels]

    acc = (pred == labels).mean()
    if epoch % 100 == 0:
      print(f"Time Taken: {time.time()-start_time:.3f}s, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.numpy():.5f}, Accuracy: {acc:.5f}")


Time Taken: 0.252s, Epoch [1/2000], Loss: 2.30259, Accuracy: 0.07812
Time Taken: 0.233s, Epoch [101/2000], Loss: 1.60009, Accuracy: 0.81250
Time Taken: 0.352s, Epoch [201/2000], Loss: 1.31239, Accuracy: 0.82812
Time Taken: 0.342s, Epoch [301/2000], Loss: 1.25362, Accuracy: 0.84375
Time Taken: 0.363s, Epoch [401/2000], Loss: 1.19342, Accuracy: 0.87500
Time Taken: 0.352s, Epoch [501/2000], Loss: 1.10693, Accuracy: 0.85938
Time Taken: 0.211s, Epoch [601/2000], Loss: 1.13220, Accuracy: 0.79688
Time Taken: 0.250s, Epoch [701/2000], Loss: 1.06580, Accuracy: 0.85938
Time Taken: 0.235s, Epoch [801/2000], Loss: 1.01031, Accuracy: 0.95312
Time Taken: 0.214s, Epoch [901/2000], Loss: 0.97295, Accuracy: 0.92188
Time Taken: 0.230s, Epoch [1001/2000], Loss: 1.06278, Accuracy: 0.85938
Time Taken: 0.216s, Epoch [1101/2000], Loss: 0.92644, Accuracy: 0.92188
Time Taken: 0.206s, Epoch [1201/2000], Loss: 0.90607, Accuracy: 0.96875
Time Taken: 0.248s, Epoch [1301/2000], Loss: 0.98052, Accuracy: 0.90625
Time

In [ ]:
# set training flag to false
Tensor.training = False

st = time.perf_counter()
avg_acc = 0
for step in range(1000):
  # random sample a batch
  samp = np.random.randint(0, X.shape[0], size=(64))
  batch = Tensor((X[samp].astype('float32') / 255.0), requires_grad=False)
  # get the corresponding labels
  labels = y[samp]

  # forward pass
  out = net.forward(batch)

  # calculate accuracy
  pred = np.argmax(out.numpy(), axis=-1)

  labels = [eval(label) for label in labels]
  avg_acc += (pred == labels).mean()

print(f"Test Accuracy: {avg_acc / 1000}")
print(f"Time Taken To Test: {time.perf_counter() - st}")

Test Accuracy: 0.93803125
Time Taken To Test: 74.64441574900002


In [ ]:
# first we need the state dict of our model
state_dict = get_state_dict(net)

# then we can just save it to a file
safe_save(state_dict, "/content/model.safetensors")

### Get Model weights & Bias

In [ ]:
## weight/bias values into the float-32 format

def list_to_string_val(flatten_list):
  return " ".join(str(item) for item in flatten_list)


def get_float_list(ndim_list):
    value_list = list(ndim_list.numpy())
    return value_list


def get_weight_bias_file(model_state_dict):
  for key, value in model_state_dict.items():
    print(f"{key}: {value.shape}, {value[0].shape}")
    model_value_list = np.array(get_float_list(value)).flatten()
    list_to_string = list_to_string_val(model_value_list)
    with open('/content/weight_bias_model.txt','ab') as f:
        f.write(str.encode(list_to_string))
        f.write(str.encode("\n"))

In [ ]:
# ### weight/bias values into the int format

# def float32_to_int(num):
#     scaled_num = int(round(num * 1000))
#     return scaled_num


# def list_to_string_val(flatten_list):
#     int_list = [float32_to_int(item) for item in flatten_list]  # Convert float values to integers
#     return " ".join(str(item) for item in int_list)


# def get_float_list(ndim_list):
#     value_list = list(ndim_list.numpy())
#     return value_list


# def get_weight_bias_file(model_state_dict):
#     for key, value in model_state_dict.items():
#         model_value_list = np.array(get_float_list(value)).flatten()
#         list_to_string = list_to_string_val(model_value_list)
#         with open('/content/weight_bias_model_int.txt', 'ab') as f:
#             f.write(str.encode(list_to_string))
#             f.write(str.encode("\n"))

### Model Inference

In [ ]:
!wget https://i.stack.imgur.com/VChE0.png

--2023-07-04 05:58:59--  https://i.stack.imgur.com/VChE0.png
Resolving i.stack.imgur.com (i.stack.imgur.com)... 146.75.32.193
Connecting to i.stack.imgur.com (i.stack.imgur.com)|146.75.32.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 307274 (300K) [image/png]
Saving to: ‘VChE0.png’

VChE0.png           100%[===================>] 300.07K  --.-KB/s    in 0.004s  

2023-07-04 05:58:59 (69.3 MB/s) - ‘VChE0.png’ saved [307274/307274]



In [ ]:
import tinygrad.tensor as tensor
from PIL import Image
# Step 1: Preprocess the image
# Your preprocessing code here
image = Image.open('/content/tinygrad/VChE0.png')
image = image.resize((28, 28))  # Resize the image to match the input size of the model

# Step 2: Load the trained model
state_dict = safe_load("/content/model.safetensors")
get_weight_bias_file(state_dict)

checkpoints = {
    'conv1.weight': state_dict['conv1.weight'],
    'conv1.bias': state_dict['conv1.bias'],
    'conv2.weight': state_dict['conv2.weight'],
    'conv2.bias': state_dict['conv2.bias'],
    'conv3.weight': state_dict['conv3.weight'],
    'conv3.bias': state_dict['conv3.bias'],
    'fc1.weight': state_dict['fc1.weight'],
    'fc1.bias': state_dict['fc1.bias'],
    'fc2.weight': state_dict['fc2.weight'],
    'fc2.bias': state_dict['fc2.bias']
}

model = LeNetTinyNet()

load_state_dict(model, checkpoints)

image = image.convert("L")

image = np.array(image)

image = image.astype(np.float32) / 255.0  # Normalize pixel values to the range [0, 1]

start_time = time.time()
preprocessed_image = image.reshape((1, 1, 28, 28))

# Step 3: Forward pass
input_tensor = tensor.Tensor(preprocessed_image)  # Create a tensor from the preprocessed image
output_tensor = net.forward(input_tensor)  # Perform forward pass

print(f"\n\nTime Taken to Predict the class: {time.time() - start_time: .4f}")

# Step 4: Obtain predictions
predicted_class= np.argmax(output_tensor.numpy(), axis=-1)
print("Predicted class:", predicted_class)

conv1.weight: (6, 1, 5, 5), (1, 5, 5)
conv1.bias: (6,), ()
conv2.weight: (16, 6, 5, 5), (6, 5, 5)
conv2.bias: (16,), ()
conv3.weight: (120, 16, 5, 5), (16, 5, 5)
conv3.bias: (120,), ()
fc1.weight: (84, 120), (120,)
fc1.bias: (84,), ()
fc2.weight: (10, 84), (84,)
fc2.bias: (10,), ()


ram used:  0.30 GB, fc2.bias                                          : 100%|██████████| 10/10 [00:00<00:00, 321.41it/s]

loaded weights in 38.39 ms, 0.30 GB loaded at 7.92 GB/s


Time Taken to Predict the class:  0.0229
Predicted class: [2]


### Compare the Feature-Map of image

In [ ]:
def list_to_string_val(flatten_list):
  return " ".join(str(item) for item in flatten_list)

def float32_to_int(num):
    scaled_num = int(round(num * 1000))
    return scaled_num

# def list_to_string_val(flatten_list):
#     int_list = [float32_to_int(item) for item in flatten_list]  # Convert float values to integers
#     return " ".join(str(item) for item in int_list)


def extract_feature_map(x):
  feature_map = x.numpy().flatten()
  feature_map_string_values = list_to_string_val(feature_map)

  with open('/content/feature_map_test.txt','ab') as f:
          f.write(str.encode(feature_map_string_values))
          f.write(str.encode("\n"))

In [ ]:
class LeNetTinyNet:
    def __init__(self):
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = x.reshape((x.shape[0], 1, 28, 28))
        extract_feature_map(x) # 0
        x = self.conv1(x)
        extract_feature_map(x) # 1
        x = x.tanh()
        extract_feature_map(x) # 2

        x = x.avg_pool2d(2)    ## Conv1:  (1, 6, 14, 14)
        extract_feature_map(x) # 3

        x = self.conv2(x)
        extract_feature_map(x)   # 4
        # print("shape of conv2: ", x.shape)
        x = x.tanh()
        extract_feature_map(x) # 5

        x = x.avg_pool2d(2)    ## Conv2:  (1, 16, 5, 5)
        extract_feature_map(x) # 6

        x = self.conv3(x)
        extract_feature_map(x)  # 7
        x = x.tanh()
        # print("shape of conv3: ", x.shape)
        extract_feature_map(x) # 8

        x = x.reshape((x.shape[0], -1))  ## Reshape:  (1, 120)
        extract_feature_map(x) # 9

        x = self.fc1(x)  ## FC1:  (1, 84)
        extract_feature_map(x) # 10
        x = x.tanh()
        extract_feature_map(x) # 11

        x = self.fc2(x)   ## FC2:  (1, 10)
        extract_feature_map(x) # 12
        return x.log_softmax()

checkpoints = {
    'conv1.weight': state_dict['conv1.weight'],
    'conv1.bias': state_dict['conv1.bias'],
    'conv2.weight': state_dict['conv2.weight'],
    'conv2.bias': state_dict['conv2.bias'],
    'conv3.weight': state_dict['conv3.weight'],
    'conv3.bias': state_dict['conv3.bias'],
    'fc1.weight': state_dict['fc1.weight'],
    'fc1.bias': state_dict['fc1.bias'],
    'fc2.weight': state_dict['fc2.weight'],
    'fc2.bias': state_dict['fc2.bias']
}

model = LeNetTinyNet()
load_state_dict(model, checkpoints)

ram used:  0.30 GB, fc2.bias                                          : 100%|██████████| 10/10 [00:00<00:00, 531.33it/s]

loaded weights in 25.53 ms, 0.30 GB loaded at 11.91 GB/s


In [ ]:
import numpy as np

file_path = "/content/mnist_test.csv"

data = np.loadtxt(file_path, delimiter=",")

X_test = data[:, 1:]
y_test = data[:, 0]

X_test = X_test / 255.0

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


In [ ]:
avg_acc, correct_pred, false_pred = 0, 0, 0
batch_size = 128

num_batches = int(np.ceil(len(X_test) / batch_size))
for i in tqdm(range(num_batches)):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(X_test))

    batch_images = X_test[start_idx:end_idx].reshape((-1, 1, 28, 28))
    input_tensor = Tensor(batch_images)  # Convert the batch of preprocessed images to a tensor

    output_tensor = model.forward(input_tensor)
    predicted_classes = np.argmax(output_tensor.numpy(), axis=-1)

    actual_labels = y_test[start_idx:end_idx].astype(int)

    avg_acc += np.sum(predicted_classes == actual_labels)
    correct_pred += np.sum(predicted_classes == actual_labels)
    false_pred += np.sum(predicted_classes != actual_labels)

avg_acc /= len(X_test)

print("Accuracy of Prediction: ", avg_acc)
print("True Positive Value: ", correct_pred)
print("False Negative Value: ", false_pred)

100%|██████████| 79/79 [01:51<00:00,  1.42s/it]

Accuracy of Prediction:  0.9427
True Positive Value:  9427
False Negative Value:  573


In [ ]:
img = [
        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84,
        185,159,151,60,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,222,254,254,254,254,241,198,198,198,198,
        198,198,198,198,170,52,0,0,0,0,0,0,0,0,0,0,0,0,67,114,72,114,163,227,254,225,254,254,254,250,229,254,
        254,140,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,66,14,67,67,67,59,21,236,254,106,0,0,0,0,0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,0,0,0,0,0,0,83,253,209,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,233,255,83,
        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,129,254,238,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
        0,0,0,0,59,249,254,62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,133,254,187,5,0,0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,205,248,58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,126,254,182,
        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,75,251,240,57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
        0,0,0,19,221,254,166,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,203,254,219,35,0,0,0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,0,0,0,0,0,0,0,38,254,254,77,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,224,254,115,
        1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,133,254,254,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
        0,0,0,61,242,254,254,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,254,254,219,40,0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,254,207,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
        0,0,0,0,0,0,0,0,0,0,0,0,0,0

]

In [ ]:
img = np.array(img)
image = img.astype(np.float32)/255.0
# print(image)
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

preprocessed_image = image.reshape((1, 1, 28, 28))

In [ ]:
Tensor.training = False
input_tensor = Tensor(preprocessed_image)  # Create a tensor from the preprocessed image
output_tensor = model.forward(input_tensor)
predicted_class= np.argmax(output_tensor.numpy(), axis=-1)

np.set_printoptions(formatter={'float_kind':'{:f}'.format})
np.set_printoptions(threshold=sys.maxsize)
print("Predicted class:", predicted_class)

Predicted class: [7]
